In [2]:
!pip install tensorflow

In [1]:
import os
import cv2
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, concatenate
from sklearn.model_selection import train_test_split

**Load and preprocess the IMDB dataset for RNN (Text Sentiment Analysis)**

In [86]:
# Parameters
vocab_size = 20000
max_len = 500
embedding_dim = 128
lstm_units = 128

In [3]:
# Load IMDB dataset
(X_train_text, y_train_text), (X_test_text, y_test_text) = imdb.load_data(num_words=vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Pad sequences
X_train_text = pad_sequences(X_train_text, maxlen=max_len)
X_test_text = pad_sequences(X_test_text, maxlen=max_len)

In [5]:
# Convert labels to categorical (positive or negative sentiment)
y_train_text = to_categorical(y_train_text, 2)
y_test_text = to_categorical(y_test_text, 2)

In [6]:
# Define the model
model = Sequential()

In [7]:
# Convert word indices to dense vectors of fixed size
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
# Processes sequences and learns temporal patterns
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))

In [9]:
# Fully connected layer for classification
model.add(Dense(2, activation='softmax'))

In [10]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# Train the Model
history = model.fit(X_train_text, y_train_text, batch_size=32, epochs=5, validation_data=(X_test_text, y_test_text))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 987s 1s/step - accuracy: 0.8063 - loss: 0.4360 - val_accuracy: 0.8358 - val_loss: 0.3970
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 1101s 1s/step - accuracy: 0.8680 - loss: 0.3236 - val_accuracy: 0.8433 - val_loss: 0.3692
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 1075s 1s/step - accuracy: 0.8908 - loss: 0.2748 - val_accuracy: 0.8475 - val_loss: 0.3749
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 978s 1s/step - accuracy: 0.9270 - loss: 0.1895 - val_accuracy: 0.8499 - val_loss: 0.3885
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 983s 1s/step - accuracy: 0.9485 - loss: 0.1373 - val_accuracy: 0.8541 - val_loss: 0.4283


In [13]:
model.save('imdb_lstm_model.h5')

with open('imdb_preprocessed_data.pkl', 'wb') as f:
    pickle.dump((X_train_text, y_train_text, X_test_text, y_test_text), f)

In [14]:
from google.colab import files

# Download the model file
files.download('imdb_lstm_model.h5')
files.download('imdb_preprocessed_data.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Load and Preprocess the FER-2013 Dataset (CNN)**

In [69]:
from google.colab import files
uploaded = files.upload()

Saving FER-2013.zip to FER-2013.zip


In [70]:
import zipfile

zip_file = "FER-2013.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('FER-2013')

In [71]:
import os
os.listdir('FER-2013')

['test', 'train']

In [72]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [73]:
# Path to the dataset folders
train_dir = 'FER-2013/train'
test_dir = 'FER-2013/test'

In [88]:
img_width, img_height = 48, 48
batch_size = 32
num_classes = 7

In [89]:
# Data augmentation and normalization for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [90]:
# Data normalization for validation set
test_datagen = ImageDataGenerator(rescale=1./255)

In [91]:
# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


In [92]:
# Load and preprocess the testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [42]:
# Define the CNN model
model = Sequential()

In [43]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [44]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
# Set up checkpoint callback to save the model
checkpoint = ModelCheckpoint('fer2013_cnn_model.keras', monitor='val_loss', save_best_only=True, mode='min')

In [46]:
steps_per_epoch = 10000 // 32

In [47]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = test_generator.samples // batch_size

In [48]:
print(f"Number of batches in train_generator: {len(train_generator)}")
print(f"Number of batches in test_generator: {len(test_generator)}")

Number of batches in train_generator: 898
Number of batches in test_generator: 225


In [49]:
train_steps = train_generator.samples // batch_size
val_steps = test_generator.samples // batch_size

In [51]:
# Train the model with the checkpoint callback
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=15,
    validation_data=test_generator,
    validation_steps=val_steps,
    callbacks=[checkpoint]
)

Epoch 1/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 155s 172ms/step - accuracy: 0.2801 - loss: 1.7570 - val_accuracy: 0.4014 - val_loss: 1.5348
Epoch 2/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 79us/step - accuracy: 0.3438 - loss: 1.5998 - val_accuracy: 0.5000 - val_loss: 1.7043
Epoch 3/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 150s 166ms/step - accuracy: 0.3817 - loss: 1.5836 - val_accuracy: 0.4647 - val_loss: 1.4145
Epoch 4/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 91us/step - accuracy: 0.4375 - loss: 1.3388 - val_accuracy: 0.4000 - val_loss: 1.3086
Epoch 5/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 200s 165ms/step - accuracy: 0.4414 - loss: 1.4629 - val_accuracy: 0.4743 - val_loss: 1.3599
Epoch 6/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3438 - loss: 1.5931 - val_accuracy: 0.3000 - val_loss: 1.4140
Epoch 7/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 151s 168ms/step - accuracy: 0.4690 - loss: 1.4031 - val_accuracy: 0.4911 - val_loss: 1.3220
Epoch 8/15
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 37us/step - accuracy: 0.6250 - loss: 1.1348

In [52]:
model.save('fer2013_cnn_model.keras')
from google.colab import files

# Download the model file
files.download('fer2013_cnn_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>